In [ ]:
%matplotlib notebook

import math
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import re
import datetime as dt

import unidecode

# enable progress bar on long operations
from tqdm.auto import tqdm
tqdm.pandas()

from collections import Counter

In [ ]:
today = pd.datetime.now().strftime("%Y%m%d")
today

In [ ]:
%%time

nan_values = ['-1.#IND', '1.#QNAN', '1.#IND', '-1.#QNAN', '#N/A N/A', '#N/A', 'N/A', 'n/a', # 'NA' is sometimes name
              '<NA>', '#NA', 'NULL', 'null', 'NaN', '-NaN', 'nan', '-nan', '']

dtypes_rciv = {
        'NUMERO_CEDULA': str,
        'NOMBRES_APELLIDOS': str,
        'GENERO': 'category',
        'FECHA_NACIMIENTO': str,
        'ESTADO_CIVIL': 'category',
        'NACIONALIDAD': 'category',
        'NOMBRES_APELLIDOS_CONYUGUE': str,
        'FECHA_DEFUNCION': str,
        'NUMERO_CEDULA_ORIGINAL': str,
        'DIGITO_VERIFICADOR': str,
        'DESCRIPCION_ESTADO_CIVIL': 'category',
        'CODIGO_CIUDADANIA': str,
        'DIA_NACIMIENTO': int,
        'MES_NACIMIENTO': int,
        'ANIO_NACIMIENTO': int,
        'CODIGO_LUGAR_NACIMTO_ORIGINAL': str,
        'CODIGO_LUGAR_NACIMIENTO': str,
        'CODIGO_INSTRUCCION': str,
        'CODIGO_PROFESION': str,
        'NOMBRES_PADRE': str,
        'CODIGO_NACIONALIDAD_PADRE': 'category',
        'NOMBRES_MADRE': str,
        'CODIGO_NACIONALIDAD_MADRE': 'category',
        'DIA_MATRIMONIO': float,
        'MES_MATRIMONIO': float,
        'ANIO_MATRIMONIO': float,
        'DIA_FALLECIMIENTO': float,
        'MES_FALLECIMIENTO': float,
        'ANIO_FALLECIMIENTO': float,
        'NUMERO_CEDULA_CONYUGE': str,
        'NUMERO_CEDULA_MADRE': str,
        'NUMERO_CEDULA_PADRE': str,
        'GENERO_LEY': str,
        'FECHA_INSC_GENERO': str,
        'COD_PARR_INSC_GENERO': str,
    }

usecols_rciv = [
    'NUMERO_CEDULA', 'NOMBRES_APELLIDOS', 'GENERO', 'FECHA_NACIMIENTO', 
#    'ESTADO_CIVIL', 
    'NACIONALIDAD', 
    'NOMBRES_APELLIDOS_CONYUGUE', 'FECHA_DEFUNCION', 'NUMERO_CEDULA_ORIGINAL', 
#    'DIGITO_VERIFICADOR', 
    'DESCRIPCION_ESTADO_CIVIL', 
#    'CODIGO_CIUDADANIA', 
#    'DIA_NACIMIENTO', 'MES_NACIMIENTO', 'ANIO_NACIMIENTO', 
#    'CODIGO_LUGAR_NACIMTO_ORIGINAL', 'CODIGO_LUGAR_NACIMIENTO', 'CODIGO_INSTRUCCION', 'CODIGO_PROFESION', 
    'NOMBRES_PADRE', #'CODIGO_NACIONALIDAD_PADRE', 
    'NOMBRES_MADRE', #'CODIGO_NACIONALIDAD_MADRE', 
    'DIA_MATRIMONIO', 'MES_MATRIMONIO', 'ANIO_MATRIMONIO', 
#    'DIA_FALLECIMIENTO', 'MES_FALLECIMIENTO', 'ANIO_FALLECIMIENTO', 
    'NUMERO_CEDULA_CONYUGE', 'NUMERO_CEDULA_MADRE', 'NUMERO_CEDULA_PADRE', 
#    'GENERO_LEY', 'FECHA_INSC_GENERO', 'COD_PARR_INSC_GENERO'
]

rf = pd.read_csv('s3://hobsons-datascience-sagemaker/dan/tmp/RCIVIL23082018.txt', sep='\t', encoding='latin',
                parse_dates=['FECHA_NACIMIENTO', 'FECHA_DEFUNCION'], dtype=dtypes_rciv, usecols=usecols_rciv,
                 keep_default_na=False, na_values=nan_values,
#                 nrows=1000000,
                )
rf.rename(columns={'NOMBRES_APELLIDOS':'nombre', 'NOMBRES_PADRE':'nombre_padre', 'NOMBRES_MADRE':'nombre_madre',
                   'FECHA_NACIMIENTO':'dt_birth', 'FECHA_DEFUNCION':'dt_death', 'GENERO':'gender',
                   'NUMERO_CEDULA':'cedula', 'NUMERO_CEDULA_ORIGINAL':'orig_cedula',
                   'DESCRIPCION_ESTADO_CIVIL':'marital_status', 'NOMBRES_APELLIDOS_CONYUGUE':'nombre_spouse',
                   "NACIONALIDAD":'nationality',
                  },
         inplace=True)
print("# records loaded :", len(rf))
#

In [5]:
# fix null Cedula data
rf['ced_spouse'] = rf.NUMERO_CEDULA_CONYUGE.replace('0', "")
rf['ced_padre'] = rf.NUMERO_CEDULA_PADRE.replace('0', "")
rf['ced_madre'] = rf.NUMERO_CEDULA_MADRE.replace('0', "")

rf.drop(columns=['NUMERO_CEDULA_CONYUGE', 'NUMERO_CEDULA_PADRE', 'NUMERO_CEDULA_MADRE'], inplace=True)

## Sanity-checks : dates

In [6]:
# this should require .loc[], right?  But it seems to be fine without...
dt_mat = rf[rf.DIA_MATRIMONIO.notnull() & rf.MES_MATRIMONIO.notnull() & rf.ANIO_MATRIMONIO.notnull()]
rf['dt_marriage'] = pd.to_datetime(dt_mat.ANIO_MATRIMONIO * 10000 
                                   + dt_mat.MES_MATRIMONIO*100 + dt_mat.DIA_MATRIMONIO, format='%Y%m%d')

# delete redundant date/time columns
rf.drop(columns=['DIA_MATRIMONIO', 'MES_MATRIMONIO', 'ANIO_MATRIMONIO'], inplace=True)

rf.loc[rf.dt_marriage >= dt.datetime(2019,1,1), 'dt_marriage'] = np.nan

In [7]:
rf['dt_birth'] = pd.to_datetime(rf.dt_birth, errors='coerce')
rf.loc[rf.dt_birth >= dt.datetime(2019,1,1), 'dt_birth'] = np.nan

rf['dt_death'] = pd.to_datetime(rf.dt_death, errors='coerce')
rf.loc[rf.dt_death >= dt.datetime(2019,1,1), 'dt_death'] = np.nan

In [8]:
rf.loc[rf.dt_birth >= dt.datetime(2019,1,1), 'dt_birth'] = np.nan

In [ ]:
rf.dtypes

In [ ]:
f3, ax3 = plt.subplots()

by_year = rf.groupby(rf.dt_birth.dt.year).cedula.count()

ax3.plot(by_year.index, by_year);

In [ ]:
fig4, ax4 = plt.subplots(figsize=(10,10))

ax4.set_ylabel('year of birth');
ax4.set_xlabel('year of death');

sub = rf[rf.dt_birth.notnull() & rf.dt_death.notnull()].sample(frac=0.08)
ax4.plot(sub.dt_death, sub.dt_birth, '.', alpha=0.1)

In [ ]:
fig, ax = plt.subplots()

byear = 1900
death_years = rf[rf.dt_birth.dt.year == byear].groupby(rf.dt_death.dt.year).cedula.count()
ax.plot(death_years.index, death_years, 'r--', label=str(byear));

byear = 1901
death_years = rf[rf.dt_birth.dt.year == byear].groupby(rf.dt_death.dt.year).cedula.count()
ax.plot(death_years.index, death_years, 'b-', label=str(byear));

byear = 1899
death_years = rf[rf.dt_birth.dt.year == byear].groupby(rf.dt_death.dt.year).cedula.count()
ax.plot(death_years.index, death_years, 'g:', label=str(byear));

byear = 1902
death_years = rf[rf.dt_birth.dt.year == byear].groupby(rf.dt_death.dt.year).cedula.count()
ax.plot(death_years.index, death_years, 'm:', label=str(byear));

ax.legend();

In [ ]:
print("# bad birth dates   : {0}".format(rf.dt_birth.isnull().sum()))
print("# time-travellers   : {0}".format(len(rf[rf.dt_death < rf.dt_birth])))

## Sanity-checks : marriages

In [ ]:
print("# zombie unions     : {0}".format(len(rf[rf.dt_marriage > rf.dt_death])))
print("# firstborn pledged : {0}".format(len(rf[(rf.dt_marriage.dt.year - rf.dt_birth.dt.year) < 10])))

rf[rf.nombre_spouse.notnull() & ~rf.marital_status.isin({"CASADO", "VIUDO", "UNION LIBRE"})]   # 1k 

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))

ax.set_ylabel('year of birth')
ax.set_xlabel('year of marriage')

# guessing that age for marriage was changed to 18, law taking effect July 2015
# BINGO: https://www.girlsnotbrides.org/child-marriage/ecuador/
sub = rf[rf.dt_marriage.notnull()].sample(frac=0.1)
ax.plot(sub.dt_marriage, sub.dt_birth, '.', alpha=0.1);